In [1]:
import os
from tqdm import tqdm
from metrics import calPSNR, calSSIM

size_map = {
    "2160P": "3840x2160",
    "1080P": "1920x1080",
    "720P": "1280x720",
    "540P": "960x540",
    "360P": "640x360",
}
enc_root  = "/hdd/YoutubeUGC/enc_rlts/vvenc"
orig_root = "/hdd/YoutubeUGC/scenes" 

In [ ]:
""" 之前把 yuv420p 删掉了 -> 从 yuv420p10le 转回去 """
# for size in sizes:
#     seqs = os.listdir(os.path.join(orig_root, 'yuv420p10le', size))
#     os.makedirs(f"{orig_root}/yuv420p/{size}", exist_ok=True)
    
#     for seq in seqs:
#         os.system(f"ffmpeg -s {size_map[size]} -pix_fmt yuv420p10le -i {os.path.join(orig_root, 'yuv420p10le', size, seq)} -pix_fmt yuv420p {os.path.join(orig_root, 'yuv420p', size, seq)} &")

In [8]:
""" 看一下为什么 540P 总有问题 """
size = "540P"
seqs = os.listdir(f"{enc_root}/{size}")

for seq in seqs:
    seq_dir = os.path.join(f"{enc_root}/{size}", seq)
    rec_dir = os.path.join(seq_dir, "rec")

    metrics_dir = os.path.join(seq_dir, "metrics")
    os.makedirs(f"{metrics_dir}/psnr", exist_ok=True)
    os.makedirs(f"{metrics_dir}/ssim", exist_ok=True)
    
    for rec_yuv in tqdm(os.listdir(rec_dir), desc=f"size:{size}, seq:{seq}"):
        orig_path = os.path.join(orig_root, size, rec_yuv.split("_qp")[0] + ".yuv")
        rec_path = os.path.join(rec_dir, rec_yuv)
        width, height = size_map[size].split("x")[0], size_map[size].split("x")[1]
        
        psnr_log = os.path.join(metrics_dir, "psnr", os.path.split(rec_path)[-1].replace(".yuv", ".txt"))
        
        cmd = f"ffmpeg -s {width}x{height} -pix_fmt yuv420p10le -i {rec_path} -s {width}x{height} -pix_fmt yuv420p -i {orig_path} -lavfi psnr='stats_file={psnr_log}' -f null -"
        os.system(cmd)

size:540P, seq:Lecture-1033: 100%|██████████| 63/63 [00:22<00:00,  2.86it/s]


#### 1. 计算 psnr 和 ssim

In [12]:
import os

sizes = ["360P", "540P", "720P", "1080P"]
for size in sizes[:1]:
    seqs = os.listdir(f"{enc_root}/{size}")
    width, height = size_map[size].split("x")[0], size_map[size].split("x")[1]

    for seq in seqs[:1]:
        seq_dir = os.path.join(f"{enc_root}/{size}", seq)
        rec_dir = os.path.join(seq_dir, "rec")

        metrics_dir = os.path.join(seq_dir, "metrics")
        os.makedirs(f"{metrics_dir}/psnr", exist_ok=True)
        os.makedirs(f"{metrics_dir}/ssim", exist_ok=True)

        for rec_yuv in tqdm(os.listdir(rec_dir)[:1], desc=f"size:{size}, seq:{seq}"):     
            # rec:  Lecture-003a_1080P_scene0_qp22_faster.yuv
            orig_path = os.path.join(orig_root, "yuv420p", size, rec_yuv.split("_qp")[0] + ".yuv")
            rec_path = os.path.join(rec_dir, rec_yuv)
            
            if all(len(os.listdir(os.path.join(metrics_dir, metric))) == len(os.listdir(rec_dir)) for metric in ["psnr", "ssim"]):
                continue
            
            calPSNR(
                orig_path, rec_path, psnr_dir=os.path.join(metrics_dir, "psnr"),
                orig_fmt="yuv420p", rec_fmt="yuv420p10le", height=height, width=width
            )
            calSSIM(
                orig_path, rec_path, ssim_dir=os.path.join(metrics_dir, "ssim"),
                orig_fmt="yuv420p", rec_fmt="yuv420p10le", height=height, width=width
            )


size:360P, seq:Lecture-1033: 100%|██████████| 1/1 [00:00<00:00, 2150.93it/s]


#### 2. 计算 vmaf

In [2]:
import subprocess
def convertfmt(
        input_path,
        output_path,
        width,
        height,
        input_fmt="yuv420p",
        output_fmt="yuv420p10le"
):
    cmd = [
        "ffmpeg",
        "-y",
        "-s", f"{width}x{height}",
        "-pix_fmt", input_fmt,
        "-i", input_path,
        "-c:v", "rawvideo",
        "-pix_fmt", output_fmt,
        output_path
    ]
    try:
        subprocess.run(cmd, text=True, capture_output=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error Conversion: {e}")

In [4]:
# 2. 计算 vmaf
# 2.1 把 origseq 都转成 yuv420p10le
import os
from glob import glob

sizes = ["720P", "1080P"]

for size in sizes:
    seqs = glob(os.path.join(orig_root, "yuv420p", size, "*.yuv"))
    
    for seq in tqdm(seqs):
        width, height = size_map[size].split("x")[0], size_map[size].split("x")[1]
        os.makedirs(f"{orig_root}/yuv420p10le/{size}", exist_ok=True)
        converted_seq = seq.replace("yuv420p", "yuv420p10le")
        
        convertfmt(seq, converted_seq, width, height)

100%|██████████| 298/298 [22:27<00:00,  4.52s/it]  


In [3]:
import json
def calVMAF(
        orig_path,
        rec_path,
        vmaf_dir,
        test_script = "/home/zhaoy/vmaf/python/vmaf/script/run_vmaf.py",
        out_fmt  = "json",
        out_file = None,
        pix_fmt  = "yuv420p10le",
        height = 270,
        width  = 480
):
    cmd = ["python", test_script, pix_fmt, f"{width}", f"{height}", orig_path, rec_path, "--out-fmt", out_fmt]
    env = os.environ.copy()
    env["PYTHONPATH"] = "python"

    try:
        rlt = subprocess.run(cmd, cwd=vmaf_dir, env=env, text=True, capture_output=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running VMAF script: {e}")
        return -1

    if out_file and out_fmt == "json":
        data = json.loads(rlt.stdout)
        with open(out_file, "w") as f:
            json.dump(data, f, indent=4)
    else:
        print(f"STDOUT: {rlt.stdout}")

In [5]:
# 2.2 计算 vmaf
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed
vmaf_dir = "/home/zhaoy/vmaf"

sizes = ["1080P"]
for size in sizes:
    seqs = os.listdir(f"{enc_root}/{size}")
    
    with ThreadPoolExecutor(max_workers=32) as executor:
        futures = []
        
        for seq in tqdm(seqs):
            seq_dir = os.path.join(f"{enc_root}/{size}", seq)
            rec_dir = os.path.join(seq_dir, "rec")
            os.makedirs(f"{seq_dir}/metrics/vmaf", exist_ok=True)
            
            for rec_yuv in os.listdir(rec_dir):
                width, height = size_map[size].split("x")[0], size_map[size].split("x")[1]
                orig_path = os.path.join(orig_root, "yuv420p10le", size, rec_yuv.split("_qp")[0] + ".yuv")
                rec_path  = os.path.join(rec_dir, rec_yuv)
                
                futures.append(
                    executor.submit(
                    calVMAF, 
                    orig_path, 
                    rec_path, 
                    vmaf_dir, 
                    "/home/zhaoy/vmaf/python/vmaf/script/run_vmaf.py", 
                    "json", 
                    os.path.join(f"{seq_dir}/metrics/vmaf", rec_yuv.replace(".yuv", ".json")),
                    "yuv420p10le", 
                    height, 
                    width
                    )
                )
        
        for futures in tqdm(as_completed(futures), total=len(futures)):
            futures.result()

  1%|          | 68/6258 [10:01<17:44:16, 10.32s/it]